# <font style="color:purple">Download the Dataset</font> 

**[Download the Vehicle registration plate](https://www.dropbox.com/s/k81ljpmzy3fgtx9/Dataset.zip?dl=1)**


Download the Vehicle Registration Plate dataset from [here](https://www.dropbox.com/s/k81ljpmzy3fgtx9/Dataset.zip?dl=1) and unzip it. 

We will have the following directory structure:

```
Dataset
├── train
│   └── Vehicle registration plate
│       └── Label
└── validation
    └── Vehicle registration plate
        └── Label
```

Unzipping the file will give you a directory `Dataset`. This directory has two folder `train` and `validation`. Each train and validation folder has `Vehicle registration plate`  folder with `.jpg` images and a folder `Labels`.  `Labels` folder has bounding box data for the images.


For example,
For image: `Dataset/train/Vehicle registration plate/bf4689922cdfd532.jpg`
Label file is  `Dataset/train/Vehicle registration plate/Label/bf4689922cdfd532.txt`

There are one or more lines in each `.txt` file. Each line represents one bounding box.
For example,
```
Vehicle registration plate 385.28 445.15 618.24 514.225
Vehicle registration plate 839.68 266.066462 874.24 289.091462
```

We have a single class detection (`Vehicle registration plate detection`) problem. So bounding box details start from the fourth column in each row.

Representation is in `xmin`, `ymin`, `xmax`, and `ymax` format.

**It has `5308` training and `386` validation dataset.**

Data is downloaded from [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html)

In [22]:
import os
import torch
import glob
import cv2

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F

from torchvision import datasets, transforms

import pandas as pd

import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks

import albumentations as A
from albumentations.pytorch import ToTensorV2


os.environ['KMP_DUPLICATE_LIB_OK']='True'


## Dataset

In [23]:
class PlateDataset(torch.utils.data.Dataset):
    
    def __init__(self, root, transforms = None, train:bool = True):
        self.root = root
        self.transforms = transforms
        
        if train:
            self.images_dir = os.path.join(root,'train','Vehicle registration plate')
        else:
            self.images_dir = os.path.join(root,'validation','Vehicle registration plate')
        self.labels_dir = os.path.join(self.images_dir, 'Label')        
        
        self.images = list(sorted(glob.glob(f'{self.images_dir}/*.jpg')))
        self.labels = list(sorted(glob.glob(f'{self.labels_dir}/*.txt')))
        
        assert len(self.images) == len( self.labels), 'Images and Labels samples dont match' 
        
        
    def __getitem__(self, idx):
        #image = read_image(self.images[idx])
        image = cv2.imread(self.images[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        df = pd.read_csv(self.labels[idx], header=None, sep=' ')
        
        num_objs = df.shape[0]
        #labels = torch.ones((num_objs,), dtype=torch.int64)
        
        #img = tv_tensors.Image(image)
        img = image
        
        boxes = []
        labels = []
        areas = []
        for r in range(df.shape[0]):
            boxes.append([df.iloc[0][3], df.iloc[0][4], df.iloc[0][5], df.iloc[0][6]])
            labels.append([1])
            areas.append((boxes[r][3] - boxes[r][1]) * (boxes[r][2] - boxes[r][0]))
                    
        #boxes = df.iloc[:, 3:]        
        #area = (boxes.iloc[:, 3] - boxes.iloc[:, 1]) * (boxes.iloc[:, 2] - boxes.iloc[:, 0])        
        
        target = {}
        target["boxes"] = boxes # tv_tensors.BoundingBoxes(boxes.values, format="XYXY", canvas_size=F.get_size(F.to_tensor(img)))
        target["labels"] = labels
        target["image_id"] = idx
        target["area"] = areas # area.values
        
       
        if self.transforms is not None:
            #print('image', type(img), len(ll), ll)
            transformed = self.transforms(image=img, bboxes=target["boxes"], class_labels=class_labels)
            img = transformed['image']
            target["boxes"] = transformed['bboxes']
            
        return img, target
    
    def __len__(self):
        return len(self.images)


In [24]:
a = PlateDataset('Dataset', None, True)

In [25]:
aa, bb = a.__getitem__(0)

In [26]:
for key, value in bb.items():
    print(type(key), type(value))
    print(key, value, '\n')    

<class 'str'> <class 'list'>
boxes [[471.04, 602.667714, 513.28, 623.140672], [471.04, 602.667714, 513.28, 623.140672]] 

<class 'str'> <class 'list'>
labels [[1], [1]] 

<class 'str'> <class 'int'>
image_id 0 

<class 'str'> <class 'list'>
area [864.7777459199968, 864.7777459199968] 



In [27]:
bb['area']

[864.7777459199968, 864.7777459199968]

### Auxiliars

In [28]:
class_labels  = [1]

In [29]:
def image_preprocess_transforms():
    transform = A.Compose([
        A.LongestMaxSize(max_size=1024),
        A.PadIfNeeded(),
        #A.HorizontalFlip(p=0.5),
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))
    
    return transform

In [30]:
#def collate_fn(data):
#    return tuple(zip(*data))

In [31]:
a = PlateDataset('Dataset', image_preprocess_transforms(), True)

In [32]:
for i in range(3):
    images, targets = a.__getitem__(i)
    pred_boxes = targets["boxes"].long()
    
    output_image = draw_bounding_boxes(images, pred_boxes,  colors="red")    
    
    plt.figure(figsize=(12, 12))
    plt.imshow(output_image.permute(1, 2, 0))

    

AssertionError: 